In [1]:
setwd("../../../")
pacman::p_load(dplyr, stringr, ComplexHeatmap)

In [2]:
table = read.csv("codes/AI/output/cross_validation/score_pred.csv", header = T, row.names = 1)
sample_meta = read.csv("data/metadata/Metadata_061523.csv", header = T, row.names = 1)
sample_meta = sample_meta[rownames(table),]
healthy_table = table[sample_meta$study_ptorhc == 'Control',]
patient_table = table[sample_meta$study_ptorhc == 'MECFS',]

In [3]:
score_meta = str_split_fixed(colnames(table),"\\.",3) %>% as.data.frame()
rownames(score_meta) = colnames(table)
colnames(score_meta) = c('model','questionnaire','score')

In [4]:
healthy_table_list = {}
patient_table_list = {}
for(score in unique(score_meta$score)){
    table_i=healthy_table[,score_meta$score == score]
    colnames(table_i) = str_split_fixed(colnames(table_i),"\\.",3)[,1]
    table_i = table_i[,c('true','immune','specie','metabolome','kegg','quest','omics')]
    healthy_table_list[[score]] = table_i
    
    table_i=patient_table[,score_meta$score == score]
    colnames(table_i) = str_split_fixed(colnames(table_i),"\\.",3)[,1]
    table_i = table_i[,c('true','immune','specie','metabolome','kegg','quest','omics')]
    patient_table_list[[score]] = table_i

}

# Plot full density in Supplimental

In [6]:
pdf("output/AI_figure/ScoreDensity_Healthy_full.pdf", width = 18, height = 2.5)
heatmap_list <- list()

for (i in 1:11) {
    table_list = healthy_table_list
    table_i = table_list[[i]]
    colnames(table_i) = c("True","Immune","Species","KEGG","Metabolome","Quest","Omics")
    title = str_replace_all(names(table_list)[[i]],"\\."," ")
    title = str_replace_all(title,"Role Limitations due to ","")
    title = str_replace_all(title," ","\n")
    title = str_replace_all(title,"_","\n")
    heatmap_i = densityHeatmap(table_i,show_column_names = F, 
               ylab=character(0),tick_label_gp = gpar(fontsize = 0), 
               quantile_gp = gpar(fontsize = 0),show_quantiles = T,
               title = title, title_gp = gpar(fontsize = 14, fontface = "bold"))
    heatmap_list[[i]] <- heatmap_i
}


table_i = table_list[[12]]
colnames(table_i) = c("True","Immune","Species","KEGG","Metabolome","Quest","Omics")
title = names(table_list)[[12]]
heatmap_i = densityHeatmap(table_i,show_column_names = F, 
               ylab=character(0),tick_label_gp = gpar(fontsize = 0), 
               quantile_gp = gpar(fontsize = 8),show_quantiles = T,
               title = title, title_gp = gpar(fontsize = 14, fontface = "bold"))
heatmap_list[[12]] <- heatmap_i

# Draw the combined heatmap with a specific layout (2 rows and 5 columns)
draw(Reduce('+', heatmap_list), background = "transparent",
     row_title = "Healthy", row_title_gp = gpar(fontsize = 14, fontface = "bold"),
     ht_gap = unit(-7, "mm"))
dev.off()

png 
  2

In [7]:
pdf("output/AI_figure/ScoreDensity_Patient_full.pdf", width = 18, height = 2.5)
heatmap_list <- list()

for (i in 1:11) {
    table_list = patient_table_list
    table_i = table_list[[i]]
    colnames(table_i) = c("True","Immune","Species","KEGG","Metabolome","Quest","Omics")
    heatmap_i = densityHeatmap(table_i,
               show_column_names = T, 
               column_names_gp = gpar(fontsize = 8, fontface = "bold"),column_names_rot = 45,
               ylab=character(0),tick_label_gp = gpar(fontsize = 0), 
               quantile_gp = gpar(fontsize = 0),show_quantiles = T,
               title = character(0))
    heatmap_list[[i]] <- heatmap_i
}


table_i = table_list[[12]]
colnames(table_i) = c("True","Immune","Species","KEGG","Metabolome","Quest","Omics")
heatmap_i = densityHeatmap(table_i,
               show_column_names = T, 
               column_names_gp = gpar(fontsize = 8, fontface = "bold"),column_names_rot = 45,
               ylab=character(0),tick_label_gp = gpar(fontsize = 0), 
               quantile_gp = gpar(fontsize = 8),show_quantiles = T,
               title = character(0))
heatmap_list[[12]] <- heatmap_i

# Draw the combined heatmap with a specific layout (2 rows and 5 columns)
draw(Reduce('+', heatmap_list), background = "transparent",
     row_title = "         Patient", row_title_gp = gpar(fontsize = 14, fontface = "bold"),
     ht_gap = unit(-7, "mm"))
dev.off()

png 
  2

# Plot example for main plot

In [8]:
score_example = c('Role.Limitations.due.to.Physical.Health','Social.Functioning','Pain')
omics_example = c("true","immune","specie","kegg","metabolome","omics")

In [9]:
healthylist = healthy_table_list[score_example]
patientlist = patient_table_list[score_example]

In [10]:
for(i in healthylist){
    print(mean(i$true))
}
for(i in patientlist){
    print(mean(i$true))
}

[1] 0.2682927
[1] 0.07698171
[1] 0.1128049
[1] 3.850877
[1] 0.6876218
[1] 0.5240375


In [11]:
pdf("output/AI_figure/Score_Density_Main.pdf", width = 8.5, height = 4)
heatmap_list <- list()

for (i in 1:3) {
    table_list = healthylist
    table_i = table_list[[i]]
    table_i = table_i[,omics_example]
    colnames(table_i) = c("True","Immune","Species","KEGG","Metabolome","Omics")
    heatmap_i = densityHeatmap(table_i,
               show_column_names = T, 
               column_names_gp = gpar(fontsize = 8, fontface = "bold"),column_names_rot = 45,
               ylab=character(0),tick_label_gp = gpar(fontsize = 0), 
               quantile_gp = gpar(fontsize = 0),show_quantiles = T,
               title = 'Healthy', title_gp = gpar(fontsize = 14))
    heatmap_list[[2*i-1]] <- heatmap_i
    
    table_list = patientlist
    table_i = table_list[[i]]
    table_i = table_i[,omics_example]
    colnames(table_i) = c("True","Immune","Species","KEGG","Metabolome","Omics")
    heatmap_i = densityHeatmap(table_i,
               show_column_names = T, 
               column_names_gp = gpar(fontsize = 8, fontface = "bold"),column_names_rot = 45,
               ylab=character(0),tick_label_gp = gpar(fontsize = 0), 
               quantile_gp = gpar(fontsize = 0),show_quantiles = T,
               title = 'ME/CFS', title_gp = gpar(fontsize = 14))
    heatmap_list[[2*i]] <- heatmap_i
}

heatmap_i = densityHeatmap(table_i,
               show_column_names = T, 
               column_names_gp = gpar(fontsize = 8, fontface = "bold"),column_names_rot = 45,
               ylab=character(0),tick_label_gp = gpar(fontsize = 0), 
               quantile_gp = gpar(fontsize = 8),show_quantiles = T,
               title = 'ME/CFS', title_gp = gpar(fontsize = 14))
heatmap_list[[6]] <- heatmap_i

draw(Reduce('+', heatmap_list), background = "transparent",
     column_title = "Physical Health      Social Functioning              Pain               ",
     column_title_gp = gpar(fontsize = 18,fontface = "bold"),
     ht_gap = unit(-7, "mm"))
dev.off()

png 
  2